In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FeedForwardClassifier(nn.Module):
    def __init__(self, input_size: int, output_size: int, hidden_sizes: list[int] = [64, 64], dropout: float = 0.1):
        super().__init__()
        layers = []
        in_features = input_size

        for hidden in hidden_sizes:
            layers.append(nn.Linear(in_features, hidden))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_features = hidden

        layers.append(nn.Linear(in_features, output_size))  # logits
        self.net = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.net(x)  # логиты — для CrossEntropyLoss

In [2]:
from blocksnet.machine_learning.strategy.torch.tensor.supervised.classification.multiclass_strategy import TorchTensorMulticlassClassificationStrategy
from sklearn.preprocessing import StandardScaler

strategy = TorchTensorMulticlassClassificationStrategy(FeedForwardClassifier, scalers={'x': StandardScaler()})

In [3]:
import pandas as pd

blocks = pd.read_pickle('blocks.pickle')
blocks = blocks[~blocks.land_use.isna()].copy()

In [4]:
from blocksnet.enums import LandUse

blocks['label'] = blocks['land_use'].apply(lambda lu : list(LandUse).index(lu))

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(blocks[['site_area']].values, blocks[['label']].values)

In [6]:
strategy.train(x_train, x_test, y_train, y_test, 10)

Train loss: 1.27816 | Test loss: 1.27394: 100%|██████████| 10/10 [00:03<00:00,  2.50it/s]


([1.3497165902968375,
  1.2920999238567967,
  1.2865896895184494,
  1.2847304327696698,
  1.2829170537434416,
  1.2820705191880328,
  1.2816825739799007,
  1.2822297786787358,
  1.2789691398220677,
  1.2781579760911828],
 [1.2839627265930176,
  1.2791556119918823,
  1.282345175743103,
  1.2730352878570557,
  1.2769335508346558,
  1.2734344005584717,
  1.2757028341293335,
  1.2759768962860107,
  1.2711817026138306,
  1.2739437818527222])

In [7]:
strategy.predict(x_test)

2025-07-11 06:25:14.421 | WARNING  | blocksnet.machine_learning.strategy.torch.tensor.supervised.classification.multiclass_strategy:predict:22 - One should provide criterion_cls. Using CrossEntropyLoss by default.


array([0, 0, 0, ..., 0, 0, 0], shape=(2313,))

In [8]:
strategy.predict_proba(x_test)

2025-07-11 06:25:14.433 | WARNING  | blocksnet.machine_learning.strategy.torch.tensor.supervised.classification.multiclass_strategy:predict_proba:28 - One should provide criterion_cls. Using CrossEntropyLoss by default.


array([[0.6121711 , 0.03393683, 0.1777652 , ..., 0.11729082, 0.01031452,
        0.00617049],
       [0.57938004, 0.03706232, 0.19173896, ..., 0.11400812, 0.01300374,
        0.0090794 ],
       [0.65725034, 0.02700835, 0.15168944, ..., 0.13420957, 0.00614444,
        0.00260797],
       ...,
       [0.43769178, 0.0314565 , 0.25461876, ..., 0.09116086, 0.02583959,
        0.0386055 ],
       [0.64665407, 0.02973446, 0.15970649, ..., 0.12484921, 0.00742672,
        0.00361911],
       [0.41008446, 0.02322978, 0.28356695, ..., 0.08099861, 0.02569893,
        0.04792282]], shape=(2313, 7), dtype=float32)